# Import Libraries

In [55]:
import requests, json
import pandas as pd
import sqlalchemy as sa


# Pull Dataset from API

We prepare a get request by first constructing th eapi url that we are going to query. We add a limit of 100000 (since the default is 100) and the resource id of the dataset. We also add headers to the request and then query the api.

Since it is an Open Api there is no authentication required. Only the headers are required so that the api does not think we are a bot hitting the endpoint

In [56]:
resource_id='2925c1ea-8313-4e9b-bdea-344e521d2298'
Nrecords=100000
api_url = 'https://data.ontario.ca/api/3/action/datastore_search?resource_id={}&limit={}'.format(resource_id, Nrecords)

In [57]:
headers = {'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.5 Safari/605.1.15'}

In [58]:
response = requests.get(api_url, headers=headers)
response.json()

{'help': 'https://data.ontario.ca/api/3/action/help_show?name=datastore_search',
 'success': True,
 'result': {'include_total': True,
  'limit': 100000,
  'records_format': 'objects',
  'resource_id': '2925c1ea-8313-4e9b-bdea-344e521d2298',
  'total_estimation_threshold': None,
  'records': [{'_id': 1,
    'Year/Année': '1940',
    'Month/Mois': 'August/août',
    'Municipality/Municipalité': 'TORONTO',
    'Totals / Totaux': 6},
   {'_id': 2,
    'Year/Année': '1940',
    'Month/Mois': 'October/octobre',
    'Municipality/Municipalité': 'TORONTO',
    'Totals / Totaux': 6},
   {'_id': 3,
    'Year/Année': '1942',
    'Month/Mois': 'November/novembre',
    'Municipality/Municipalité': 'TORONTO',
    'Totals / Totaux': 6},
   {'_id': 4,
    'Year/Année': '1944',
    'Month/Mois': 'April/avril',
    'Municipality/Municipalité': 'TORONTO',
    'Totals / Totaux': 6},
   {'_id': 5,
    'Year/Année': '1944',
    'Month/Mois': 'January/janvier',
    'Municipality/Municipalité': 'TORONTO',
   

This dataset is of the death registrations in Ontario by location. It has columns for year, month, location and number of deaths for that month and location

In [59]:
data = response.json()
data.keys()


dict_keys(['help', 'success', 'result'])

In [60]:
deaths = pd.DataFrame(data['result']['records'])
deaths

,_id,Year/Année,Month/Mois,Municipality/Municipalité,Totals / Totaux
0,1,1940,August/août,TORONTO,6
1,2,1940,October/octobre,TORONTO,6
2,3,1942,November/novembre,TORONTO,6
3,4,1944,April/avril,TORONTO,6
4,5,1944,January/janvier,TORONTO,6
...,...,...,...,...,...
68048,68049,2023,December/décembre,WHITBY,37
68049,68050,2023,December/décembre,WHITCHURCH-STOUFFVILLE,16
68050,68051,2023,December/décembre,WINDSOR,269
68051,68052,2023,December/décembre,WOODSTOCK,50


# Data Cleaning and Feature Engineering

In [61]:
# Data Cleaning
deaths['Month'] = deaths['Month/Mois'].apply(lambda x: x.split('/')[0])
deaths.rename(columns={'Year/Année': 'Year'}, inplace=True)
deaths['Year'] = deaths['Year'].astype(int)

# We want information of deaths for Ontario by year
deaths_by_year = deaths.groupby('Year').agg(Total=('Totals / Totaux', 'sum'))
deaths_by_year.reset_index(inplace=True)

# We calculate five year rolling data and start the rolling calculation from 1944 since 1943 and 1942 data is missing
deaths_by_year['Five Year Rolling Average'] = deaths_by_year[deaths_by_year['Year'] > 1944]['Total'].rolling(5).mean().round(2)

# We also calculate total deaths since 1940
deaths_by_year['Total Deaths Since 1940'] = deaths_by_year['Total'].cumsum()

# For each year we also get the month with the most deaths and the number of these deaths
# This is a slightly more involved process
deaths_by_year_and_month = deaths.groupby(['Year', 'Month']).agg(Total=('Totals / Totaux', 'sum'))
deaths_by_year_and_month.reset_index(inplace=True)

deaths_by_year_and_highest_month = deaths_by_year_and_month.loc[deaths_by_year_and_month.groupby('Year')['Total'].idxmax()]
deaths_by_year_and_highest_month.reset_index(inplace=True)
deaths_by_year_and_highest_month.drop('index', inplace=True, axis=1)

deaths_by_year_and_highest_month.rename(columns={'Total': 'Number of Deaths in Highest Month', 'Month': 'Month with most Deaths'}, inplace=True)

deaths_by_year = pd.merge(deaths_by_year, deaths_by_year_and_highest_month, on='Year')

deaths_by_year

,Year,Total,Five Year Rolling Average,Total Deaths Since 1940,Month with most Deaths,Number of Deaths in Highest Month
0,1940,12,NaN,12,August,6
1,1942,6,NaN,18,November,6
2,1944,12,NaN,30,April,6
3,1945,13,NaN,43,January,7
4,1946,22,NaN,65,September,9
...,...,...,...,...,...,...
77,2019,103463,97289.0,3046365,January,9633
78,2020,111574,101428.4,3157939,April,10815
79,2021,113469,105503.8,3271408,January,11247
80,2022,121697,110305.2,3393105,January,12699


# Create Database Engine and Connection
## Login with York Credentials

In [62]:
york_username = "dali96"
# Dictionary - params to set up the DB Engine
mmai_db_secret = {
    "database": "dali96_db",
    "drivername": "mssql+pyodbc",
    "host": "mmai2024-ms-sql-server.c1oick8a8ywa.ca-central-1.rds.amazonaws.com",
    "port": "1433",
    "username": york_username,
    "password": "2024!Schulich",
    "driver": "ODBC Driver 18 for SQL Server",
    "trust_cert": "yes"
}


# Function to create the DB engine - DO NOT TOUCH
def create_db_engine(db_config):
    connection_url = sa.engine.URL.create(
        drivername=db_config.get("drivername"),
        username=db_config.get("username"),
        password=db_config.get("password"),
        host=db_config.get("host"),
        port=db_config.get("port"),
        database=db_config.get("database"),
        query={
            "driver": db_config.get("driver"),
            "TrustServerCertificate": db_config.get("trust_cert")
        }
    )
    return sa.create_engine(connection_url)


# Initialize the MySql Engine using SQL Alchemy
mssql_engine = create_db_engine(mmai_db_secret)

# Ingest Data into our Database

In [63]:
deaths_by_year.to_sql(
    name   = 'ontario_deaths',
    con    = mssql_engine,
    schema = 'uploads',
    if_exists = 'replace',
    index  = False,
    dtype  = {
        'year'      : sa.types.DATE,
        'total' : sa.types.INTEGER,
        'five_year_rolling_average' : sa.types.DECIMAL(10,2),
        'total_deaths_since_1940': sa.types.INTEGER,
        'month_with_most_deaths': sa.types.String,
        'number_of_deaths_in_month': sa.types.INTEGER
    },
    method = 'multi'
)

82